# ElaraNet

In [43]:
import tensorflow as tf
# Hide GPU from visible devices
tf.config.set_visible_devices([], 'GPU')
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
import scipy.optimize as optimize
from datetime import datetime
from random import randint
import numpy as np
import os
import plotly.graph_objects as go

Loading data from dataset:

In [44]:
data = np.loadtxt("sigma_data.csv")

In [45]:
sigma = data[0]
e_req = abs(data[1])
zeta = sigma / e_req

In [46]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sigma, y=e_req, mode="lines"))
fig.update_layout(title='Absolute value of energy requirements over sigma', xaxis_title=r"$\sigma$", yaxis_title=r"$E_{\mathrm{req}}$", autosize=True)

fig.show()

In [48]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sigma, y=1/sigma, mode="lines"))
fig.update_layout(title='Shell wall thickness over sigma', autosize=True,xaxis_title=r"$\sigma$", yaxis_title="thickness")

fig.show()

We want the shell wall thickness to be minimized (that is, to have maximum internal volume), and the energy requirement to be minimized as well. Therefore, we want to maximise $\sigma$ and minimize $E_{\mathrm{req.}}$, as given below:

$$
\zeta = \frac{\sigma}{|E_{\mathrm{req.}}|}
$$

In [11]:
# normalizes data to between 0 and 1
def rescale(a, min_val=0, max_val=1):
    """
    Takes an arbritrary array and normalizes data to between
    0 and 1 for better (and faster) training
    """
    return np.interp(a, (a.min(), a.max()), (min_val, max_val))

In [12]:
def unscale_val(val, arr):
    """
    Takes a value X from a normalized (0,1) range to the range
    of an array arr
    """
    new_range = arr.max() - arr.min()
    return (val * new_range) + arr.min()

unscale = lambda sigma: unscale_val(sigma, zeta)

In [13]:
# Set up training data
x_train = sigma
y_train = rescale(zeta)

In [16]:
# Set up and train the neural network
model = Sequential()
model.add(Input(shape=(1,)))
model.add(Dense(50, activation='tanh', name="dense_1"))
model.add(Dense(1, activation='tanh', name="dense_2"))
model.compile(optimizer='sgd', loss='mse', metrics=['mse'])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 50)                100       
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [19]:
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [20]:
epochs = 5000

train_history = model.fit(x_train, y_train, batch_size=8, epochs=epochs, callbacks=[tensorboard_callback])

Epoch 1/5000
250/250 [==============================] - 1s 1ms/step - loss: 0.0268 - mse: 0.0268
Epoch 2/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0033 - mse: 0.0033
Epoch 3/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0031 - mse: 0.0031
Epoch 4/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0029 - mse: 0.0029
Epoch 5/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0028 - mse: 0.0028
Epoch 6/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0027 - mse: 0.0027
Epoch 7/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0027 - mse: 0.0027
Epoch 8/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0026 - mse: 0.0026
Epoch 9/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0025 - mse: 0.0025
Epoch 10/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0025 - mse: 0.0025
Epoch 11/5000
250/250 [======

250/250 [==============================] - 0s 1ms/step - loss: 0.0014 - mse: 0.0014
Epoch 168/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0014 - mse: 0.0014
Epoch 169/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 170/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 171/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 172/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 173/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 174/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 175/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 176/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 177/5000
250/250 [=

250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 332/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 333/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 334/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0013 - mse: 0.0013
Epoch 335/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 336/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 337/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 338/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011   
Epoch 339/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 340/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 341/5000
250/250

250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 496/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 497/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 498/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 499/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 500/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 501/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 502/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 503/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 504/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012
Epoch 505/5000
250/250 [=

250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 660/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 661/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011    
Epoch 662/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 663/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0010 - mse: 0.0010
Epoch 664/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0010 - mse: 0.0010 
Epoch 665/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 666/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011
Epoch 667/5000
250/250 [==============================] - 0s 963us/step - loss: 9.8891e-04 - mse: 9.8891e-04
Epoch 668/5000
250/250 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011  
Epoch 66

...

250/250 [==============================] - 0s 1ms/step - loss: 2.3132e-04 - mse: 2.3132e-04
Epoch 4449/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.2640e-04 - mse: 2.2640e-04
Epoch 4450/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.3607e-04 - mse: 2.3607e-04
Epoch 4451/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1692e-04 - mse: 2.1692e-04
Epoch 4452/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.2793e-04 - mse: 2.2793e-04
Epoch 4453/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1977e-04 - mse: 2.1977e-04
Epoch 4454/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.4682e-04 - mse: 2.4682e-04
Epoch 4455/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.3423e-04 - mse: 2.3423e-04
Epoch 4456/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1378e-04 - mse: 2.1378e-04
Epoch 4457/5000
250/250 [===================

250/250 [==============================] - 0s 1ms/step - loss: 2.1166e-04 - mse: 2.1166e-04
Epoch 4525/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.3101e-04 - mse: 2.3101e-04
Epoch 4526/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.2429e-04 - mse: 2.2429e-04
Epoch 4527/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.3971e-04 - mse: 2.3971e-04
Epoch 4528/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.3141e-04 - mse: 2.3141e-04
Epoch 4529/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.5369e-04 - mse: 2.5369e-04
Epoch 4530/5000
250/250 [==============================] - 0s 993us/step - loss: 2.0510e-04 - mse: 2.0510e-04
Epoch 4531/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1085e-04 - mse: 2.1085e-04
Epoch 4532/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.4668e-04 - mse: 2.4668e-04
Epoch 4533/5000
250/250 [=================

250/250 [==============================] - 0s 1ms/step - loss: 2.3749e-04 - mse: 2.3749e-04
Epoch 4675/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.4390e-04 - mse: 2.4390e-04
Epoch 4676/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.2846e-04 - mse: 2.2846e-04
Epoch 4677/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.2466e-04 - mse: 2.2466e-04
Epoch 4678/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.0815e-04 - mse: 2.0815e-04
Epoch 4679/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1865e-04 - mse: 2.1865e-04
Epoch 4680/5000
250/250 [==============================] - 0s 1ms/step - loss: 1.9977e-04 - mse: 1.9977e-04
Epoch 4681/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1184e-04 - mse: 2.1184e-04
Epoch 4682/5000
250/250 [==============================] - 0s 984us/step - loss: 2.2530e-04 - mse: 2.2530e-04
Epoch 4683/5000
250/250 [=================

250/250 [==============================] - 0s 1ms/step - loss: 2.0429e-04 - mse: 2.0429e-04
Epoch 4825/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1302e-04 - mse: 2.1302e-04
Epoch 4826/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.2133e-04 - mse: 2.2133e-04
Epoch 4827/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1292e-04 - mse: 2.1292e-04
Epoch 4828/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.3566e-04 - mse: 2.3566e-04
Epoch 4829/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.3656e-04 - mse: 2.3656e-04
Epoch 4830/5000
250/250 [==============================] - 0s 1ms/step - loss: 1.8417e-04 - mse: 1.8417e-04
Epoch 4831/5000
250/250 [==============================] - 0s 1ms/step - loss: 1.9303e-04 - mse: 1.9303e-04
Epoch 4832/5000
250/250 [==============================] - 0s 1ms/step - loss: 1.9626e-04 - mse: 1.9626e-04
Epoch 4833/5000
250/250 [===================

250/250 [==============================] - 0s 1ms/step - loss: 2.2858e-04 - mse: 2.2858e-04
Epoch 4975/5000
250/250 [==============================] - 0s 999us/step - loss: 2.0299e-04 - mse: 2.0299e-04
Epoch 4976/5000
250/250 [==============================] - 0s 994us/step - loss: 2.1415e-04 - mse: 2.1415e-04
Epoch 4977/5000
250/250 [==============================] - 0s 986us/step - loss: 2.0953e-04 - mse: 2.0953e-04
Epoch 4978/5000
250/250 [==============================] - 0s 1ms/step - loss: 1.8483e-04 - mse: 1.8483e-04
Epoch 4979/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1827e-04 - mse: 2.1827e-04
Epoch 4980/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.2519e-04 - mse: 2.2519e-04
Epoch 4981/5000
250/250 [==============================] - 0s 989us/step - loss: 2.0715e-04 - mse: 2.0715e-04
Epoch 4982/5000
250/250 [==============================] - 0s 1ms/step - loss: 2.1060e-04 - mse: 2.1060e-04
Epoch 4983/5000
250/250 [===========

In [21]:
y_pred = model.predict(x_train).flatten()

63/63 [==============================] - 0s 1ms/step


In [49]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sigma, y=y_train, mode="lines", name="Actual"))
fig.add_trace(go.Scatter(x=sigma, y=y_pred, mode="lines", name="NN predicted"))
fig.update_layout(title='Model predictions vs. actual values', autosize=True, xaxis_title=r"$\sigma$", yaxis_title=r"$\zeta$")

fig.show()

In [32]:
model.save('saved_model/elaranet.h5')

In [24]:
f = lambda sigma: unscale(model.predict([sigma])[0][0])

Finally, SciPy's optimizer is used to find an ideal value of $\sigma$ to balance energy requirements and internal volume:

In [89]:
def optimize_zeta():
    num = randint(0, len(sigma))
    initial_guess = [sigma[num], zeta[num]]
    result = optimize.minimize(f, initial_guess)
    if result.success:
        fitted_params = result.x
        print(fitted_params)
        return fitted_params[0]
    else:
        raise ValueError(result.message)

optim_sigmas = [optimize_zeta() for i in range(100)]

1/1 [==============================] - 0s 22ms/step
[1.63500000e+01 7.92225411e-43]
1/1 [==============================] - 0s 19ms/step
[6.9650000e+01 8.0639436e-43]
1/1 [==============================] - 0s 20ms/step
[9.60000000e+00 7.90383988e-43]
1/1 [==============================] - 0s 20ms/step
[3.44000000e+01 7.92968942e-43]
1/1 [==============================] - 0s 19ms/step
[2.760000e+01 7.928491e-43]
1/1 [==============================] - 0s 19ms/step
[4.66000000e+01 7.93067409e-43]
1/1 [==============================] - 0s 19ms/step
[4.60500000e+01 7.93064577e-43]
1/1 [==============================] - 0s 18ms/step
[4.77500000e+01 7.93073019e-43]
1/1 [==============================] - 0s 16ms/step
[6.7650000e+01 8.0379654e-43]
1/1 [==============================] - 0s 21ms/step
[1.22500000e+01 7.91471116e-43]
1/1 [==============================] - 0s 20ms/step
[6.21500000e+01 7.98918683e-43]
1/1 [==============================] - 0s 21ms/step
[3.05000000e+01 7.92910037e-43]


1/1 [==============================] - 0s 20ms/step
[1.3900000e+01 7.9185561e-43]
1/1 [==============================] - 0s 21ms/step
[3.20500000e+01 7.92936033e-43]
1/1 [==============================] - 0s 15ms/step
[4.12500000e+01 7.93034847e-43]
1/1 [==============================] - 0s 17ms/step
[2.24500000e+01 7.92676903e-43]
1/1 [==============================] - 0s 19ms/step
[8.52500000e+01 8.27957395e-43]
1/1 [==============================] - 0s 15ms/step
[6.58500000e+01 8.00063127e-43]
1/1 [==============================] - 0s 18ms/step
[4.12500000e+01 7.93034847e-43]
1/1 [==============================] - 0s 16ms/step
[6.62000000e+01 8.00156624e-43]
1/1 [==============================] - 0s 22ms/step
[6.93000000e+01 8.06297138e-43]
1/1 [==============================] - 0s 20ms/step
[2.24500000e+01 7.92676903e-43]
1/1 [==============================] - 0s 19ms/step
[8.44000000e+01 8.26278201e-43]
1/1 [==============================] - 0s 18ms/step
[9.2150000e+01 8.3604732e-

In [90]:
fig = go.Figure()
fig.add_trace(go.Box(x=optim_sigmas, boxpoints='all', name=r"$\sigma$"))
fig.update_layout(title="Optimized sigma values")
fig.show()